# Hue setup

In [1]:
import requests
discovered = requests.get('https://discovery.meethue.com/')
discovered.json()

if len(discovered.json()) == 0:
    print("No Hue found on this network :(")
else:
    internalIPAddress = discovered.json()[0]['internalipaddress']
    print(f"Found: {discovered.json()}\nConnect button on Hue before continuing")

Found: [{'id': 'ecb5fafffe0d140f', 'internalipaddress': '192.168.86.64', 'port': 443}]
Connect button on Hue before continuing


In [2]:
from huesdk import Hue
from pathlib import Path

# Ignore warnings from urllib3 about insecure http connection
import urllib3
urllib3.disable_warnings()

username_file = Path("username.txt")
username = ""

if username_file.is_file():
    with open(username_file, 'r') as f:
        username = f.read()
        f.close()
        print(f"Read username, it is '{username}'")

if username == "":
    print("Did not find cached username - connect to bridge instead")
    print("Press the 'Connect' button on Hue")
    # ... or set to: "glV5HAHnttjuOCDGtFiceHncWnU9U60yww2UaBOl"
    username = Hue.connect(bridge_ip=internalIPAddress)
    print(f"Discovered username, it is '{username}'.")
    with open(username_file, 'w') as f:
        f.write(username)
        f.close()

Read username, it is 'glV5HAHnttjuOCDGtFiceHncWnU9U60yww2UaBOl'


In [3]:
hue = Hue(bridge_ip=internalIPAddress, username=username)

lights = hue.get_lights()
rooms = hue.get_groups()
print(f"rooms: {[room.name for room in rooms]}")

--------  Top
1
--------  Spaceship
1
--------  Counter
1
--------  Low
1
--------  Pillar
1
--------  Mid
1
--------  Cabinet
1
--------  My desk
127
--------  Bram screen 
127
--------  Booth
2
--------  Lose desk strip
1
rooms: ['Kitchen', 'Hallway', 'Office', 'Bedroom', 'Upstairs', 'Battlegrounds', 'Living room', 'Downstairs']


# Set up Whisper

In [4]:
from faster_whisper import WhisperModel
model_size = "base.en"
model = WhisperModel(model_size, device="cpu", compute_type="int8")

# Set up audio

In [5]:
from ipywebrtc import AudioRecorder, CameraStream
import torchaudio
from IPython.display import Audio
import matplotlib.pyplot as plt
%matplotlib inline
import os

# String similarity search
See: https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2

In [6]:
from sentence_transformers import SentenceTransformer, util
import torch

sentencetransformer = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [7]:
commands = []

targets = lights + rooms

for target in targets:
    # Somewhat convoluted - we need a copy of target to keep it in scope
    createOnSwitch = lambda t: lambda: t.on()
    createOffSwitch = lambda t: lambda: t.off()
    
    commands.append({
        'phrase': f"Turn the {target.name} light on",
        'action': createOnSwitch(target)
    })
    commands.append({
        'phrase': f"Turn the {target.name} light off",
        'action': createOffSwitch(target)
    })

phrases = [command['phrase'] for command in commands]
embeddings = sentencetransformer.encode(phrases)
print(phrases)

['Turn the Top light on', 'Turn the Top light off', 'Turn the Spaceship light on', 'Turn the Spaceship light off', 'Turn the Counter light on', 'Turn the Counter light off', 'Turn the Low light on', 'Turn the Low light off', 'Turn the Pillar light on', 'Turn the Pillar light off', 'Turn the Mid light on', 'Turn the Mid light off', 'Turn the Cabinet light on', 'Turn the Cabinet light off', 'Turn the My desk light on', 'Turn the My desk light off', 'Turn the Bram screen  light on', 'Turn the Bram screen  light off', 'Turn the Booth light on', 'Turn the Booth light off', 'Turn the Lose desk strip light on', 'Turn the Lose desk strip light off', 'Turn the Kitchen light on', 'Turn the Kitchen light off', 'Turn the Hallway light on', 'Turn the Hallway light off', 'Turn the Office light on', 'Turn the Office light off', 'Turn the Bedroom light on', 'Turn the Bedroom light off', 'Turn the Upstairs light on', 'Turn the Upstairs light off', 'Turn the Battlegrounds light on', 'Turn the Battlegrou

# Inference

In [8]:
# jupyter nbextension enable --py widgetsnbextension
camera = CameraStream(constraints={'audio': True,'video':False})
recorder = AudioRecorder(stream=camera)
recorder

AudioRecorder(audio=Audio(value=b'', format='webm'), stream=CameraStream(constraints={'audio': True, 'video': …

In [9]:
audio_file = "audio.webm"

with open(audio_file, 'wb') as f:
    f.write(recorder.audio.value)

# Whisper
# voice to text
segments, info = model.transcribe(audio_file, beam_size=5)
s = ""
for segment in segments:
    s += segment.text    
s = s.strip()
print(f"command: {s}")

# Sententence-transformer
# Find most similar command available
needle_embedding = sentencetransformer.encode(s)
cos_sim = util.cos_sim(needle_embedding, embeddings)
print(cos_sim)
best_match_index = torch.argmax(cos_sim)
print(f"Most similar command: '{commands[best_match_index]['phrase']}'")

# Execute
commands[best_match_index]['action']()

command: Turn off the upstairs lights.
tensor([[0.5894, 0.6383, 0.4022, 0.4712, 0.4856, 0.5293, 0.5748, 0.6164, 0.4503,
         0.5032, 0.4665, 0.5223, 0.5302, 0.5613, 0.4553, 0.4921, 0.3801, 0.4309,
         0.4458, 0.5260, 0.4003, 0.4414, 0.4986, 0.5520, 0.6169, 0.6727, 0.4960,
         0.5339, 0.5391, 0.5864, 0.8158, 0.8932, 0.3195, 0.4073, 0.5452, 0.5807,
         0.7647, 0.8367]])
Most similar command: 'Turn the Upstairs light off'
